# ***NOTE:-***
The problem here is Generator's accuracy will go down, sometimes it happens with Discriminator also,  if tried to train in Tensorflow 2.0 it's more like a mode collapse and this problem occur  when you train the model from scratch  
# but if you try to use the existing trained  model it works Great   


### So if you want to train from scratch you can do so in version Tensorflow 1.14 and then save the model and then run the saved model in 2.0 for production **bold text**

In [1]:
!pip install tensorflow-gpu==2.0rc0

     |████████████████████████████████| 380.5MB 57kB/s 
     |████████████████████████████████| 4.3MB 29.2MB/s 
     |████████████████████████████████| 501kB 48.0MB/s 
     |████████████████████████████████| 71kB 30.0MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.1-cp36-none-any.whl size=58500 sha256=ea712e2a43e18ef782ef1f2373bcb17dd88ffb9ba41a53006cf4c6137f0be56b
  Stored in directory: /root/.cache/pip/wheels/91/98/8d/10e3d4e04c959597a411b91acd3695e9e2d210e68ce3427aad
Successfully built opt-einsum


In [2]:
#Loading the Data from Drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Tensorflow 2.0 Dependencies 
use these only if you want to Deploy it for production

In [0]:
#Importing Are Dependencies 
#Tensorflow 2.0 version
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image 
from tqdm import tqdm
import os

# Tensorflow 1.14 Dependencies 
use these only if you want to train the model completley from scratch 

In [ ]:
import tensorflow as tf 
from tensorflow import keras 
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image 
from tqdm import tqdm
import os

In [5]:
#Check if the Right Version is installed or not 
tf.__version__

'2.0.0-rc0'

In [0]:
#Our Hyperparameters for Images   
GENERATE_RES = 2 # (1=32, 2=64, 3=96, etc.)
GENERATE_SQUARE = 32 * GENERATE_RES # rows/cols (should be square)
IMAGE_CHANNELS = 3


#Our Hyperparameters for image Preview  
PREVIEW_ROWS = 5
PREVIEW_COLS = 7
PREVIEW_MARGIN = 16
SAVE_FREQ = 100

# Size vector to generate images from
SEED_SIZE = 100

#Model Controller Parameters
EPOCHS = 10000
BATCH_SIZE = 64

#Path  Configuration
OUTPUT = 'Path/to/Output/'

#Path to the Discriminator Model if you have
DISCRIMINATOR_MODEL = 'Path/to/Discriminator'

#Path to the Generator Model if you have
GENERATOR_MODEL = 'Path/to/Generator' 


print(f"Will generate {GENERATE_SQUARE}px square images.")


Will generate 64px square images.


In [0]:
#Loading the Np format Data File  in the Training_data variable 
numpy_training_images = 'Path/to/Numpy_data_file/'
Training_data = np.load(numpy_training_images)

In [0]:
#Building the Generator Network 
def Generator(seed_size, channels):
    model = Sequential()

    model.add(Dense(4*4*256,activation="relu",input_dim=seed_size))
    model.add(Reshape((4,4,256)))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))

    model.add(UpSampling2D())
    model.add(Conv2D(256,kernel_size=3,padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation("relu"))
   
    # Output resolution, additional upsampling
    for i in range(GENERATE_RES):
      model.add(UpSampling2D())
      model.add(Conv2D(128,kernel_size=3,padding="same"))
      model.add(BatchNormalization(momentum=0.8))
      model.add(Activation("relu"))

    # Final CNN layer
    model.add(Conv2D(channels,kernel_size=3,padding="same"))
    model.add(Activation("tanh"))

    input = Input(shape=(seed_size,))
    generated_image = model(input)

    return Model(input,generated_image)


In [0]:
#Building the Discriminator Network 
def Discriminator(image_shape):
    model = Sequential()

    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=image_shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))

    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))


    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))


    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    input_image = Input(shape=image_shape)

    validity = model(input_image)

    return Model(input_image, validity)

In [0]:
#Function for saving the images 
from PIL import Image

def save_images(cnt,noise):
  image_array = np.full(( 
      PREVIEW_MARGIN + (PREVIEW_ROWS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 
      PREVIEW_MARGIN + (PREVIEW_COLS * (GENERATE_SQUARE+PREVIEW_MARGIN)), 3), 
      255, dtype=np.uint8)
  
  generated_images = generator.predict(noise)

  generated_images = 0.5 * generated_images + 0.5

  image_count = 0
  for row in range(PREVIEW_ROWS):
      for col in range(PREVIEW_COLS):
        r = row * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        c = col * (GENERATE_SQUARE+16) + PREVIEW_MARGIN
        image_array[r:r+GENERATE_SQUARE,c:c+GENERATE_SQUARE] = generated_images[image_count] * 255
        image_count += 1

          
  output_path = os.path.join(OUTPUT)
  if not os.path.exists(output_path):
    os.makedirs(output_path)
  
  filename = os.path.join(output_path,f"train-{cnt}.png")
  im = Image.fromarray(image_array)
  im.save(filename)

In [0]:
#Preparing the model to Train
image_shape = (GENERATE_SQUARE,GENERATE_SQUARE,IMAGE_CHANNELS)
optimizer = Adam(2e-4,0.5)

#discriminator =  load_model(DISCRIMINATOR_MODEL)
discriminator = Discriminator(image_shape)
discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
#generator = load_model(GENERATOR_MODEL)
generator =  Generator(SEED_SIZE,IMAGE_CHANNELS)

random_input = Input(shape=(SEED_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input,validity)
combined.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])

In [13]:
generator.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 100)]             0         
_________________________________________________________________
sequential_1 (Sequential)    (None, 64, 64, 3)         2043011   
Total params: 2,043,011
Trainable params: 2,041,475
Non-trainable params: 1,536
_________________________________________________________________


In [14]:
discriminator.summary()

W0829 15:58:22.256295 140445732149120 training.py:2055] Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 64, 64, 3)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 1)                 6337601   
Total params: 12,671,234
Trainable params: 6,333,633
Non-trainable params: 6,337,601
_________________________________________________________________


In [0]:
#TRAINING!!!!
EPOCHS = 10000

Model_path = '/content/drive/My Drive/Zeus/Model'
y_real = np.ones((BATCH_SIZE,1))
y_fake = np.zeros((BATCH_SIZE,1))

fixed_seed = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, SEED_SIZE))

cnt = 1
for epoch in range(EPOCHS):
    idx = np.random.randint(0,Training_data.shape[0],BATCH_SIZE)
    x_real = Training_data[idx]

    # Generate some images
    seed = np.random.normal(0,1,(BATCH_SIZE,SEED_SIZE))
    x_fake = generator.predict(seed)

    # Train discriminator on real and fake
    discriminator_metric_real = discriminator.train_on_batch(x_real,y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake,y_fake)
    discriminator_metric = 0.5 * np.add(discriminator_metric_real,discriminator_metric_generated)
    
    # Train generator on Calculate losses
    generator_metric = combined.train_on_batch(seed,y_real)
    
    # Time for an update?
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_seed)
        cnt += 1
        print(f"Epoch {epoch}, Discriminator accuarcy: {discriminator_metric[1]}, Generator accuracy: {generator_metric[1]}")



MODEL = '/content/drive/My Drive/Zeus/Designer_AI/Designer_AI_Models_and_Weights/Sports_Cars '
#Models
discriminator.save(os.path.join(MODEL,"Discriminator_v2.h5"))
generator.save(os.path.join(MODEL,"Generator_v2.h5"))
#Weights
discriminator.save_weights(os.path.join(MODEL,"Discriminator_weights_v2.h5"))
generator.save_weights(os.path.join(MODEL,"Generator_weights_v2.h5"))
